In [258]:
import pandas as pd
import math
import numpy as np
pd.set_option("display.max_rows", 500, "display.max_columns", 500)

In [259]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from keras.metrics import MeanSquaredError
from sklearn.metrics import r2_score
import math

In [260]:
df = pd.read_csv('data_output.csv', low_memory=False)
df.head(10)

Neutroticism:Anxiety  Neutroticism:Angry Hostility  \
0                    37                            40   
1                    23                            47   
2                   100                            97   
3                    77                            70   
4                    97                           100   
5                    87                            53   
6                    77                            57   
7                    70                            13   
8                    50                            23   
9                    57                            27   

   Neutroticism:Depression  Neutroticism:Self-Consciousness  \
0                       53                               73   
1                       13                               23   
2                      100                              100   
3                       73                               73   
4                       80                               57   
5                       67                               50   
6                       67                               73   
7                       57                               70   
8                       40                               27   
9                       60                               53   

   Neutroticism:Impulsiveness  Neutroticism:Vulnerability  \
0                          27                          47   
1                          40                          17   
2                          57                         100   
3                          47                          77   
4                          73                          80   
5                          80                          67   
6                          60                          63   
7                          47                          63   
8                          77                          40   
9                          43                          50   

   Extraversion:Warmth  Extraversion:Gregariousness  \
0                   10                            3   
1                   17                           10   
2                   40                           10   
3                   80                           70   
4                   37                           40   
5                   73                           83   
6                   43                           50   
7                   63                           27   
8                   87                           77   
9                   23                           10   

   Extraversion:Assertiveness  Extraversion:Activity  \
0                          13                     20   
1                          63                     40   
2                          53                     57   
3                          93                     83   
4                          97                     63   
5                          63                     47   
6                          37                     43   
7                          73                     70   
8                          83                     67   
9                          27                     47   

   Extraversion:Excitement-Seeking  Extraversion:Positive Emotions  \
0                               13                              23   
1                               50                              43   
2                               53                              47   
3                               87                              87   
4                               63                              43   
5                               63                              73   
6                               60                              43   
7                               33                              80   
8                               63                              80   
9                               67                              47   

   Openness:Fan

In [261]:
cols = df.columns
categories = []
for i in range (len(cols)):
    if i == 0:
        categories.append(cols[i][:np.char.find(cols[i], ':')])
    else:
        if cols[i][:np.char.find(cols[i], ':')] != cols[i-1][:np.char.find(cols[i-1], ':')]:
            categories.append(cols[i][:np.char.find(cols[i], ':')])

categories

['Neutroticism',
 'Extraversion',
 'Openness',
 'Agreeableness',
 'Conscientiousness',
 'Novelty Seeking',
 'Harm avoidance',
 'Reward dependence',
 'Persistence',
 'Self-directedness',
 'Cooperativeness',
 'Self-transcendence',
 'Primary Trait Dimensions',
 'Folk Scales',
 'Special Purpose Scales',
 'Honesty-Humility',
 'Emotionality',
 'eXtraversion',
 'Agreeableness',
 'Concienciousness',
 'Openness to Experience',
 'Scales',
 'Extraversion',
 'Agreeableness',
 'Independence',
 'Openness to Experince',
 'Methodicalness',
 'Industriousness',
 'Adjustment',
 'Ambition',
 'Sociability',
 'Interpersonal Sensitivity',
 'Prudence',
 'Learning Approach']

In [262]:
def my_round(i):
    f = math.floor(i)
    return f if i - f < 0.5 else f+1

In [263]:
avg_list = []
temp = []
for index, row in df.iterrows():
    temp_list = []
    count = 0
    for c in cols:
        if c[:np.char.find(c, ':')] == categories[count]:
            temp_list.append(row[c])
        else:
            temp.append(my_round(np.average(temp_list)))
            temp_list = [row[c]]
            count+=1
    temp.append(my_round(np.average(temp_list)))
    avg_list.append(temp)
    temp = []


new_df = pd.DataFrame(avg_list, columns = categories)

new_df.insert(0, 'Result_ID', range(0, 0 + len(new_df)))
new_df



Result_ID  Neutroticism  Extraversion  Openness  Agreeableness  \
0             0            46            14        53             69   
1             1            27            37        61             52   
2             2            92            43        85             39   
3             3            70            83        68             44   
4             4            81            57        87             40   
...         ...           ...           ...       ...            ...   
2012       2012            75            26        67             61   
2013       2013            71            23        60             64   
2014       2014            69            78        85             63   
2015       2015            61            18        78             67   
2016       2016            52            52        78             66   

      Conscientiousness  Novelty Seeking  Harm avoidance  Reward dependence  \
0                    53               22              72                 19   
1                    62               46              38                 34   
2                    46               61              74                 60   
3                    65               65              38                 54   
4                    32               87              76                 38   
...                 ...              ...             ...                ...   
2012                 43               42              75                 37   
2013                 33               48              76                 32   
2014                 61               48              47                 74   
2015                 56               29              84                 34   
2016                 80               42              54                 54   

      Persistence  Self-directedness  Cooperativeness  Self-transcendence  \
0              28                 49               55                  30   
1              44                 65               48                  19   
2              63                 13               59                  64   
3              68                 47               54                  89   
4              48                 45               45                  67   
...           ...                ...              ...                 ...   
2012           47                 34               59                  40   
2013           35                 41               60                  66   
2014           68                 43               67                  62   
2015           52                 39               59                  49   
2016           74                 56               64                  60   

      Primary Trait Dimensions  Folk Scales  Special Purpose Scales  \
0                           39           35                      42   
1                           41           66                      68   
2                           63           31                      12   
3                           70           52                      42   
4                           54           55                      40   
...                        ...          ...                     ...   
2012                        47           40                      34   
2013                        45           37                      32   
2014                        63           59                      44   
2015                        54           42                      40   
2016                        55           53                      54   

      Honesty-Humility  Emotionality  eXtraversion  Agreeableness  \
0                   75            38             3             53   
1                   36            29            33             33   
2                   14            69            48             33   
3                    8            51            87             34   
4                   18            73            55             20   


In [264]:
data = pd.read_csv('data.csv', sep ='\t', low_memory=False)
data.head(10)

for c in data.columns:
    if c[-1:] == 'E' or  c[-1:] == 'I':
        data.drop(c, axis = 1, inplace = True)

questions = data.columns
data.insert(0, 'Response_ID', range(0, 0 + len(df)))
questions

Index(['Q1A', 'Q2A', 'Q3A', 'Q4A', 'Q5A', 'Q6A', 'Q7A', 'Q8A', 'Q9A', 'Q10A',
       ...
       'Q172A', 'Q173A', 'Q174A', 'Q175A', 'Q176A', 'Q177A', 'Q178A', 'Q179A',
       'Q180A', 'Q181A'],
      dtype='object', length=181)

In [265]:
trait = 'Persistence'
reg_df = data.merge(new_df[['Result_ID',trait]], left_on='Response_ID', right_on='Result_ID')
reg_df

Response_ID  Q1A  Q2A  Q3A  Q4A  Q5A  Q6A  Q7A  Q8A  Q9A  Q10A  Q11A  \
0               0    5    2    1    2    5    2    4    4    2     4     2   
1               1    6    2    2    6    2    2    6    3    6     5     6   
2               2    1    1    1    7    5    4    1    7    7     5     3   
3               3    2    6    6    7    2    7    2    5    6     6     7   
4               4    1    7    5    7    7    3    5    1    7     4     2   
...           ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   
2012         2012    2    5    2    2    2    1    2    6    2     5     1   
2013         2013    2    5    2    2    3    2    2    6    2     3     2   
2014         2014    1    5    6    6    3    6    3    1    4     5     4   
2015         2015    1    1    2    2    4    1    2    6    6     6     2   
2016         2016    1    5    2    6    5    3    4    5    2     5     2   

      Q12A  Q13A  Q14A  Q15A  Q16A  Q17A  Q18A  Q19A  Q20A  Q21A  Q22A  Q23A  \
0        4     6     3     1     6     2     1     4     1     7     1     4   
1        5     6     2     2     5     5     1     2     2     2     3     2   
2        7     7     3     7     7     5     1     7     7     7     4     5   
3        6     6     3     4     6     7     2     6     6     2     7     5   
4        7     1     1     7     7     5     2     6     7     1     2     3   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
2012     6     3     3     6     1     1     1     5     2     5     2     6   
2013     5     3     3     5     5     2     2     5     2     7     2     3   
2014     6     6     3     5     2     5     7     7     7     2     6     6   
2015     4     4     5     1     3     2     3     6     2     7     1     5   
2016     7     6     5     2     5     1     2     6     4     5     2     5   

      Q24A  Q25A  Q26A  Q27A  Q28A  Q29A  Q30A  Q31A  Q32A  Q33A  Q34A  Q35A  \
0        5     2     2     5     2     3     7     7     2     2     4     2   
1        5     3     2     5     5     3     6     5     3     3     3     2   
2        7     4     1     7     7     7     7     7     3     7     2     5   
3        6     7     2     7     6     7     4     1     6     6     6     6   
4        7     7     2     7     7     7     5     7     4     7     7     6   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
2012     5     6     1     5     4     3     7     6     1     6     5     4   
2013     5     3     2     5     4     4     6     5     1     6     3     5   
2014     5     4     2     6     6     7     2     2     4     6     2     4   
2015     4     6     1     4     5     7     7     7     1     2     3     1   
2016     3     5     1     7     6     4     6     6     1     2     5     2   

      Q36A  Q37A  Q38A  Q39A  Q40A  Q41A  Q42A  Q43A  Q44A  Q45A  Q46A  Q47A  \
0        1     7     1     3     2     7     6     2     2     6     2     6   
1        1     6     2     7     5     6     2     2     5     2     3     3   
2        7     7     5     7     7     5     4     7     4     1     5     1   
3        6     2     7     6     6     6     5     6     7     2     6     6   
4        4     7     6     3     5     2     1     7     7     5     7     6   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
2012     5     7     2     3     2     6     6     6     4     2     1     2   
2013     5     6     1     3     3     5     5     3     3     5     3     5   
2014     4     1     7     3     6     6     6     4     6     2     4     1   
2015     5     6     1     2     4     7     4     2     4     1     2     1   
2016     4     4     1     6     2     6     5     4     6     2     1     2   

      Q48A  Q49A  Q50A  Q51A  Q52A  Q53A  Q54A  Q55A  Q56A  Q57A  Q58A  Q59A  \
0        4     5     3     3     2     2     2     2     4     5     3     3   
1        2     2     5    

In [266]:
data = data.drop(['Response_ID'], axis = 1)
reg_df = reg_df.drop(['Response_ID', 'Result_ID'], axis = 1)
print(reg_df)
corr = reg_df.corr()
corr = corr[trait]
corr

      Q1A  Q2A  Q3A  Q4A  Q5A  Q6A  Q7A  Q8A  Q9A  Q10A  Q11A  Q12A  Q13A  \
0       5    2    1    2    5    2    4    4    2     4     2     4     6   
1       6    2    2    6    2    2    6    3    6     5     6     5     6   
2       1    1    1    7    5    4    1    7    7     5     3     7     7   
3       2    6    6    7    2    7    2    5    6     6     7     6     6   
4       1    7    5    7    7    3    5    1    7     4     2     7     1   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   ...   ...   
2012    2    5    2    2    2    1    2    6    2     5     1     6     3   
2013    2    5    2    2    3    2    2    6    2     3     2     5     3   
2014    1    5    6    6    3    6    3    1    4     5     4     6     6   
2015    1    1    2    2    4    1    2    6    6     6     2     4     4   
2016    1    5    2    6    5    3    4    5    2     5     2     7     6   

      Q14A  Q15A  Q16A  Q17A  Q18A  Q19A  Q20A  Q21A  Q22A  Q23A  Q24A  Q25

Q1A            0.136636
Q2A           -0.154709
Q3A            0.121282
Q4A            0.545465
Q5A            0.464563
Q6A            0.224945
Q7A           -0.010786
Q8A           -0.032281
Q9A           -0.014849
Q10A           0.193862
Q11A           0.137555
Q12A          -0.020824
Q13A           0.441652
Q14A           0.629117
Q15A          -0.196317
Q16A          -0.046155
Q17A          -0.033053
Q18A           0.017751
Q19A           0.065977
Q20A           0.115083
Q21A          -0.243441
Q22A           0.441221
Q23A           0.057000
Q24A          -0.106179
Q25A           0.691970
Q26A           0.007600
Q27A           0.135307
Q28A          -0.011409
Q29A          -0.185278
Q30A          -0.209572
Q31A          -0.409341
Q32A          -0.085178
Q33A          -0.207966
Q34A          -0.049321
Q35A          -0.175701
Q36A           0.046389
Q37A          -0.132845
Q38A           0.092691
Q39A           0.183381
Q40A           0.132725
Q41A           0.290992
Q42A          -0

In [267]:
high_relation_qs = []
for i in range(len(corr)-1):
    if abs(corr[i]) > 0.4:
        high_relation_qs.append(corr.index[i])

In [268]:
data[high_relation_qs]

Q4A  Q5A  Q13A  Q14A  Q22A  Q25A  Q31A  Q48A  Q81A  Q101A  Q110A  Q116A  \
0       2    5     6     3     1     2     7     4     5      2      1      3   
1       6    2     6     2     3     3     5     2     5      3      5      3   
2       7    5     7     3     4     4     7     7     3      5      6      7   
3       7    2     6     3     7     7     1     6     5      7      6      2   
4       7    7     1     1     2     7     7     1     2      7      7      4   
...   ...  ...   ...   ...   ...   ...   ...   ...   ...    ...    ...    ...   
2012    2    2     3     3     2     6     6     2     5      3      6      5   
2013    2    3     3     3     2     3     5     3     3      2      3      5   
2014    6    3     6     3     6     4     2     3     6      6      6      3   
2015    2    4     4     5     1     6     7     1     3      2      6      6   
2016    6    5     6     5     2     5     6     7     5      5      6      1   

      Q137A  Q141A  Q144A  Q145A  Q152A  Q167A  
0         2      5      3      6      6      4  
1         5      6      2      5      6      5  
2         6      7      6      7      7      7  
3         4      6      1      2      5      2  
4         6      3      1      2      6      1  
...     ...    ...    ...    ...    ...    ...  
2012      4      5      3      5      5      3  
2013      4      3      3      5      6      3  
2014      7      3      3      2      6      6  
2015      6      6      6      7      5      5  
2016      6      6      6      2      1      6  

[2017 rows x 18 columns]

In [269]:
# X = data[data.columns].to_numpy()
# y = new_df["Neutroticism"].to_numpy()
inputs = data[high_relation_qs].shape[1]
inputs

18

In [270]:
def NN_model(inputs):
	# create model
	model = Sequential()
	# model.add(Dense(181, input_dim=181, kernel_initializer='normal', activation='relu'))
	# model.add(Dense(150, activation='relu'))
	# model.add(Dense(100, activation='relu'))
	# model.add(Dense(50, activation='relu'))
	# model.add(Dense(1, kernel_initializer='normal'))


	model.add(Dense(inputs, input_dim=inputs, kernel_initializer='normal', activation='relu'))
	model.add(Dense(20, activation='relu'))
	model.add(Dense(15, activation='relu'))
	model.add(Dense(10, activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam', metrics=[MeanSquaredError()])
	return model

In [271]:
# estimators = []
# estimators.append(('standardize', StandardScaler()))
# estimators.append(('mlp', KerasRegressor(build_fn=NN_model, epochs=10, batch_size=5, verbose=0)))
# pipeline = Pipeline(estimators)
# kfold = KFold(n_splits=10)
# results = cross_val_score(pipeline, X, Y, cv=kfold)
# print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [272]:
X = data[high_relation_qs].to_numpy()
y = new_df[trait]

In [273]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [274]:
model = NN_model(inputs)

In [275]:
history = model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=[X_test, y_test])

Epoch 1/100
43/43 [==============================] - 0s 4ms/step - loss: 3183.2588 - mean_squared_error: 3183.2588 - val_loss: 3064.2957 - val_mean_squared_error: 3064.2957
Epoch 2/100
43/43 [==============================] - 0s 2ms/step - loss: 2089.5974 - mean_squared_error: 2089.5974 - val_loss: 490.0403 - val_mean_squared_error: 490.0403
Epoch 3/100
43/43 [==============================] - 0s 2ms/step - loss: 155.3261 - mean_squared_error: 155.3261 - val_loss: 101.2631 - val_mean_squared_error: 101.2631
Epoch 4/100
43/43 [==============================] - 0s 2ms/step - loss: 90.5098 - mean_squared_error: 90.5098 - val_loss: 78.6543 - val_mean_squared_error: 78.6543
Epoch 5/100
43/43 [==============================] - 0s 2ms/step - loss: 74.2940 - mean_squared_error: 74.2940 - val_loss: 64.0100 - val_mean_squared_error: 64.0100
Epoch 6/100
43/43 [==============================] - 0s 2ms/step - loss: 61.4343 - mean_squared_error: 61.4343 - val_loss: 52.4659 - val_mean_squared_error: 

In [276]:
y_predict = model.predict(X_test)

In [277]:
import matplotlib.pyplot as plt

plt.figure(figsize=(16,16))
plt.plot(range(100), y_test[:100], color="r", label="True Cooperativeness")
plt.plot(range(100), y_predict[:100], color="b", label="Predicted Cooperativeness")
plt.xlabel("Response Number")
plt.ylabel("Cooperativeness Value")
plt.legend()

In [278]:
r2_score(y_test, y_predict)

0.9772460949715919